### Traditional Model Solution

In [1]:
import math
import numpy
from gurobipy import *
import os

p = 0.8
sigma=math.sqrt(p*(1-p)/4)
mu_model = 1-p/2
h_List = [182,660,1155,465,46,218,270,345,1292,2235,2770,1350,195,2129,8697,2775,172,27,900,17201,1279,21780]
b_List = [1000,1000,1000,1000,1000,1000,1000,1000,5000,1000,5000,1000,1000,5000,10000,5000,1000,1000,1000,20000,5000,40000]
L_List = [19,15,8,9,9,6,9,8,16,15,6,7,2,7,5,10,11,35,10,8,7,4]

S_MAX = 101
N = 22
Structure = [[-1],[-1],[-1],[-1],[-1],[-1],[-1],[-1],[0,1],[-1],[2,3,4],[-1],[-1],[5,6,7],[8,9,10],[11,12],[-1],[-1],[-1],[13,14,15],[16,17],[18,19,20]]
S_in_supplier = [10,10,10,10,10,10,10,10,1e20,10,1e20,10,10,1e20,1e20,1e20,10,10,10,1e20,1e20,1e20]
S_out_customer = [1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,5]

Non_Start = [] #find the set of stages with upstream stages
for j in range (N):
    if (Structure[j][0] > -1):
        Non_Start.append(j)

for i in range (N): #re-scaling
    b_List[i] = b_List[i] *10 / 100.0
    h_List[i] = h_List[i] / 100.0
            
def optimal_solution (k):
    F_S_S = []
    solution = []
    for S_in in range (0,S_MAX):
        temp = []
        for S_out in range (0,S_MAX):
            SubCost = []
            for j in range (N):
                if ((S_in+L_List[j]-S_out) >= 0):
                    SubCost.append(h_List[j]*k*sigma*math.sqrt(S_in+L_List[j]-S_out))
                else: 
                    SubCost.append(5000)
            temp.append(SubCost)
        F_S_S.append(temp)

    F_S_S = numpy.array(F_S_S)
    print("F_S_S generated!")

    #Solve MILP
    try:
        m = Model("Algo-Trad")
        # Create variables
        chi = m.addVars(N, S_MAX, S_MAX, vtype=GRB.BINARY, name="chi")
        # Set objective
        obj = 0
        for j in range (N):
            for mu in range (S_MAX):
                for nu in range (S_MAX):
                    obj += F_S_S[mu,nu,j]*chi[j,mu,nu]
        m.setObjective(obj, GRB.MINIMIZE)
        # Add constraint
        for j in range (N):
            m.addConstr(quicksum(chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == 1)
            if (S_out_customer[j] < 1e20):
                m.addConstr(quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_out_customer[j])
            if (S_in_supplier[j] < 1e20):
                m.addConstr(quicksum(mu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_in_supplier[j])
        for i in range (len(Non_Start)):
            for j in Structure[Non_Start[i]]:
                m.addConstr(quicksum(mu*chi[Non_Start[i],mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) >= quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)))

        m.optimize()

    except GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    except AttributeError:
        print('Encountered an attribute error')

    for j in range (N):
        for mu in range (S_MAX):
            for nu in range (S_MAX):
                if(chi[j,mu,nu].X == 1):
                    solution.append([F_S_S[mu,nu,j],mu,nu]) #obj,S_in,S_out
    
    return solution.copy()

Trad_Binomial = open("Trad-Binomial-Simulation.txt", mode='w')
Trad_Poisson = open("Trad-Poisson-Simulation.txt", mode='w')
Trad_Solution = open("Trad-Solution.txt", mode='w')
for k in [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,
         2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,
          3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,
         4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5.0]:
    opt_temp = optimal_solution(k)
    opt_temp = numpy.array(opt_temp)
    S_in_List = opt_temp[:,1]
    S_out_List = opt_temp[:,2]
    S_in_List = S_in_List.tolist()
    S_out_List = S_out_List.tolist()
    W_List = S_in_List
    B_List = []
    for i in range (N):
        B_List.append(round(mu_model*(S_in_List[i]+L_List[i]-S_out_List[i])+k*sigma*math.sqrt(S_in_List[i]+L_List[i]-S_out_List[i])))        
        
    Trad_Solution.write(str(k)+'\t')
    for i in range (N):
        Trad_Solution.write(str(S_in_List[i])+'\t')
    for i in range (N):
        Trad_Solution.write(str(S_out_List[i])+'\t')
    for i in range (N):
        Trad_Solution.write(str(B_List[i])+'\t')
    Trad_Solution.write('\n') 
    #simulation-binomial
    for w in range (100):
        numpy.random.seed(w+10)
        Z = numpy.random.binomial(1,p,500).tolist() #Z-1
        demand = [] #real-time demand 0 or 1
        for z in Z:
            if (z == 0):
                demand.append(1)
            else:
                demand.append(0)
                demand.append(1)
        cost = numpy.zeros(N).tolist()
        backordertimes = numpy.zeros(N).tolist()
        for i in range (N):
            stock = []
            stock.append(B_List[i])
            for j in range (1, len(demand)):
                stock.append(stock[j-1])
                if ((j-S_out_List[i]) >= 0):
                    stock[j] -= demand[int(j-S_out_List[i])]
                if ((j-W_List[i]-L_List[i]) >= 0):
                    stock[j] += demand[int(j-W_List[i]-L_List[i])]
                if (stock[j] < 0):
                    cost[i] -= stock[j]*b_List[i]
                    backordertimes[i] -= stock[j]
                    stock[j] = 0
                else:
                    cost[i] += h_List[i]*stock[j]
        Trad_Binomial.write(str(k)+'\t'+str(w)+'\t')
        for i in range (N):
            Trad_Binomial.write(str(cost[i])+'\t')
        for i in range (N):
            Trad_Binomial.write(str(backordertimes[i])+'\t')
        Trad_Binomial.write('\n')
    print(k, "Simulation-Binomial Ended!")
    #simulation-poisson
    for w in range (100):
        numpy.random.seed(w+10)
        demand = numpy.random.poisson(mu_model,int((1-p+2*p)*500)).tolist()
        cost = numpy.zeros(N).tolist()
        backordertimes = numpy.zeros(N).tolist()
        for i in range (N):
            stock = []
            stock.append(B_List[i])
            for j in range (1, len(demand)):
                stock.append(stock[j-1])
                if ((j-S_out_List[i]) >= 0):
                    stock[j] -= demand[int(j-S_out_List[i])]
                if ((j-W_List[i]-L_List[i]) >= 0):
                    stock[j] += demand[int(j-W_List[i]-L_List[i])]
                if (stock[j] < 0):
                    cost[i] -= stock[j]*b_List[i]
                    backordertimes[i] -= stock[j]
                    stock[j] = 0
                else:
                    cost[i] += h_List[i]*stock[j]
        Trad_Poisson.write(str(k)+'\t'+str(w)+'\t')
        for i in range (N):
            Trad_Poisson.write(str(cost[i])+'\t')
        for i in range (N):
            Trad_Poisson.write(str(backordertimes[i])+'\t')
        Trad_Poisson.write('\n')
    print(k, "Simulation-Poisson Ended!")
Trad_Binomial.close() #every row: k,w,cost of stages 0~21, backorder times of stages 0~21
Trad_Poisson.close()
Trad_Solution.close() #every row: k, inbound service time of stages 0~21, outbound service time of stages 0~21, stock of stages 0~21

F_S_S generated!

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-07-13
Using license file /Users/estelle/gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0xa8c79ef5
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 632.1871077
Presolve removed 15 rows and 199464 columns (presolve time = 6s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 5.73s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 517.4343078
Variable types: 0 continuous, 23908 integer (23908


Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      6s
      85    1.7729690e+02   0.000000e+00   0.000000e+00      6s

Root relaxation: objective 1.772969e+02, 85 iterations, 0.02 seconds
Total elapsed time = 5.70s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  177.29690    0   30  620.92117  177.29690  71.4%     -    5s
H    0     0                     424.5900169  177.29690  58.2%     -    5s
H    0     0                     338.0472801  177.29690  47.6%     -    5s
H    0     0                     333.3436187  177.29690  46.8%     -    5s
     0     0  206.45547    0   34  333.34362  206.45547  38.1%     -    5s
H    0     0                     312.6350317  206.45547  34.0%     -    5s
     0     0  212.78695    0   33  312.63503  212.78695  31.9%     -    5s
     0     0  247.04354    0   57

H    0     0                     364.7408703  240.86471  34.0%     -    6s
     0     0  248.25144    0   33  364.74087  248.25144  31.9%     -    6s
     0     0  288.21746    0   57  364.74087  288.21746  21.0%     -    6s
     0     0  288.21746    0   31  364.74087  288.21746  21.0%     -    6s
     0     0  288.21746    0   49  364.74087  288.21746  21.0%     -    6s
H    0     0                     364.6461584  288.21746  21.0%     -    6s
     0     0  292.69829    0   55  364.64616  292.69829  19.7%     -    6s
     0     0  293.64954    0   53  364.64616  293.64954  19.5%     -    6s
     0     0  293.64954    0   54  364.64616  293.64954  19.5%     -    6s
     0     0  314.18159    0   39  364.64616  314.18159  13.8%     -    6s
     0     0  314.87710    0   39  364.64616  314.87710  13.6%     -    6s
     0     0  314.92494    0   40  364.64616  314.92494  13.6%     -    6s
     0     0  316.62648    0   53  364.64616  316.62648  13.2%     -    6s
H    0     0             

     0     0  334.51233    0   55  416.73847  334.51233  19.7%     -    6s
     0     0  335.59947    0   53  416.73847  335.59947  19.5%     -    6s
     0     0  335.59947    0   54  416.73847  335.59947  19.5%     -    6s
     0     0  359.06467    0   39  416.73847  359.06467  13.8%     -    6s
     0     0  359.85955    0   39  416.73847  359.85955  13.6%     -    7s
     0     0  359.91421    0   40  416.73847  359.91421  13.6%     -    7s
     0     0  361.85883    0   53  416.73847  361.85883  13.2%     -    7s
H    0     0                     368.7491276  361.85883  1.87%     -    7s
     0     0  362.68107    0   46  368.74913  362.68107  1.65%     -    7s
     0     0  362.76902    0   41  368.74913  362.76902  1.62%     -    7s
     0     0  363.59470    0   41  368.74913  363.59470  1.40%     -    7s
H    0     0                     368.0599945  363.59470  1.21%     -    7s
     0     0  363.59470    0   27  368.05999  363.59470  1.21%     -    7s
     0     0  366.39322  

     0     0  411.47269    0   26  414.06749  411.47269  0.63%     -    7s

Cutting planes:
  Cover: 3
  Clique: 6
  StrongCG: 1
  GUB cover: 14
  RLT: 1

Explored 1 nodes (503 simplex iterations) in 7.65 seconds
Thread count was 4 (of 4 available processors)

Solution count 9: 414.067 414.458 468.831 ... 1137.94

Optimal solution found (tolerance 1.00e-04)
Best objective 4.140674937919e+02, best bound 4.140674937919e+02, gap 0.0000%
1.8 Simulation-Binomial Ended!
1.8 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0xbfcb3d0a
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 

...

     0     0  838.99867    0   53 1041.84617  838.99867  19.5%     -    8s
     0     0  838.99867    0   54 1041.84617  838.99867  19.5%     -    8s
     0     0  897.66167    0   39 1041.84617  897.66167  13.8%     -    9s
     0     0  899.64887    0   39 1041.84617  899.64887  13.6%     -    9s
     0     0  899.78553    0   40 1041.84617  899.78553  13.6%     -    9s
     0     0  904.64707    0   50 1041.84617  904.64707  13.2%     -    9s
H    0     0                     921.8728191  904.64707  1.87%     -    9s
     0     0  906.70267    0   44  921.87282  906.70267  1.65%     -    9s
     0     0  906.92255    0   39  921.87282  906.92255  1.62%     -    9s
     0     0  908.98675    0   41  921.87282  908.98675  1.40%     -    9s
H    0     0                     920.1499862  908.98675  1.21%     -    9s
     0     0  908.98675    0   27  920.14999  908.98675  1.21%     -    9s
     0     0  918.31948    0    2  920.14999  918.31948  0.20%     -    9s

Cutting planes:
  Cover:

     0     0  944.63131    0   39 1093.93848  944.63131  13.6%     -    8s
     0     0  944.77481    0   40 1093.93848  944.77481  13.6%     -    8s
     0     0  949.87943    0   50 1093.93848  949.87943  13.2%     -    8s
H    0     0                     976.6161097  949.87943  2.74%     -    8s
H    0     0                     972.2281737  949.87943  2.30%     -    8s
     0     0  952.03781    0   47  972.22817  952.03781  2.08%     -    8s
     0     0  952.26868    0   39  972.22817  952.26868  2.05%     -    8s
     0     0  954.38422    0   56  972.22817  954.38422  1.84%     -    8s
H    0     0                     967.7950703  954.38422  1.39%     -    8s
     0     0  954.38422    0   22  967.79507  954.38422  1.39%     -    9s
H    0     0                     966.1574855  954.38422  1.22%     -    9s
     0     0  957.20727    0   36  966.15749  957.20727  0.93%     -    9s
     0     0  957.44909    0   35  966.15749  957.44909  0.90%     -    9s
     0     0  957.91299  

     0     0  989.76409    0   41 1146.03078  989.76409  13.6%     -    8s
     0     0  996.26753    0   45 1146.03078  996.26753  13.1%     -    8s
H    0     0                    1013.1191241  996.26753  1.66%     -    8s
     0     0  997.46175    0   35 1013.11912  997.46175  1.55%     -    8s
     0     0  997.52813    0   38 1013.11912  997.52813  1.54%     -    8s
     0     0 1000.65842    0   40 1013.11912 1000.65842  1.23%     -    8s
     0     0 1000.65842    0   25 1013.11912 1000.65842  1.23%     -    8s
H    0     0                    1012.1649848 1000.65842  1.14%     -    8s
     0     0 1010.15142    0    2 1012.16498 1010.15142  0.20%     -    8s

Cutting planes:
  Cover: 2
  Clique: 13
  MIR: 1
  StrongCG: 1
  GUB cover: 14
  RLT: 1

Explored 1 nodes (516 simplex iterations) in 8.94 seconds
Thread count was 4 (of 4 available processors)

Solution count 9: 1012.16 1013.12 1146.03 ... 2781.62

Optimal solution found (tolerance 1.00e-04)
Best objective 1.012164984825e


Optimal solution found (tolerance 1.00e-04)
Best objective 1.058172484135e+03, best bound 1.058172484135e+03, gap 0.0000%
4.6 Simulation-Binomial Ended!
4.6 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0xdc7f8b8f
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 2971.2794063
Presolve removed 15 rows and 199464 columns (presolve time = 8s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 8.61s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 2431.9412469
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simp


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      8s
      85    7.2396234e+02   0.000000e+00   0.000000e+00      8s

Root relaxation: objective 7.239623e+02, 85 iterations, 0.04 seconds
Total elapsed time = 8.05s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  723.96234    0   30 2535.42811  723.96234  71.4%     -    8s
H    0     0                    1733.7425689  723.96234  58.2%     -    8s
H    0     0                    1380.3597270  723.96234  47.6%     -    8s
H    0     0                    1361.1531095  723.96234  46.8%     -    8s
     0     0  843.02649    0   34 1361.15311  843.02649  38.1%     -    8s
H    0     0                    1276.5930459  843.02649  34.0%     -    8s
     0     0  868.88003    0   33 1276.59305  868.88003  31.9%     -    8s
     0     0

In [3]:
import os
import numpy

N = 22

File = open("Trad-Solution.txt", mode='r')
temp = []
while True:
    Line = File.readline()
    if Line:
        Line = Line[:-2]
        Line = list(map(float,Line.split('\t')))
        Line = Line[1:]
        temp.append(Line)
    else:
        break
File.close()
print("inbound service time")
print(numpy.average(temp, axis=0)[0:N])
print(numpy.max(temp, axis=0)[0:N])
print(numpy.min(temp, axis=0)[0:N])
print("outbound service time")
print(numpy.average(temp, axis=0)[N:(2*N)])
print(numpy.max(temp, axis=0)[N:(2*N)])
print(numpy.min(temp, axis=0)[N:(2*N)])
print("base stock")
print(numpy.average(temp, axis=0)[(2*N):(3*N)])
print(numpy.max(temp, axis=0)[(2*N):(3*N)])
print(numpy.min(temp, axis=0)[(2*N):(3*N)])

inbound service time
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
outbound service time
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
base stock
[ 3.56097561  0.         13.36585366 14.         14.          9.26829268
 11.31707317 10.63414634 24.53658537 14.          0.         12.
  8.6097561   0.          0.          0.         15.3902439  31.02439024
 14.         13.36585366  5.07317073  0.        ]
[ 4.  0. 15. 16. 16. 11. 13. 12. 27. 16.  0. 14. 10.  0.  0.  0. 17. 34.
 16. 15.  6.  0.]
[ 3.  0. 12. 12. 12.  8. 10.  9. 22. 12.  0. 10.